In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [3]:
y_preds = {}

In [4]:
# prepare training data


In [6]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=100, sample_weights_size=100, is_oversample=False, include_new_data=False)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, discretize_size=50, dis_strategy="quantile", normalize=True, scale=False)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=50000)
feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%Data size: 25429

Finished windowfying
Featurizing calculate_feats=True, normalize=True, discretize=True, discretize_size=50, include_feats=['first_prons', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 25429, 25429
Data size: 25611, 25611
Calculating first prons
Calculating NSSI words
Calculating first prons
Calculating NSSI words
Normalizing features
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 4 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.loc

Is the combined the same from tfidf: False


In [7]:
# traditional classifiers

In [8]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.85      0.99      0.92     21304
           1       0.79      0.17      0.28      4307

    accuracy                           0.85     25611
   macro avg       0.82      0.58      0.60     25611
weighted avg       0.84      0.85      0.81     25611

[[21109   195]
 [ 3588   719]]
{'precision': 0.6813186813186813, 'recall': 0.5961538461538461, 'F1': 0.635897435897436, 'ERDE_5': 0.26253819799119915, 'ERDE_50': 0.12922836697572548, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.6086341590481623}


In [9]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.86      0.99      0.92     21304
           1       0.78      0.20      0.31      4307

    accuracy                           0.86     25611
   macro avg       0.82      0.59      0.62     25611
weighted avg       0.84      0.86      0.82     25611

[[21059   245]
 [ 3459   848]]
{'precision': 0.6597938144329897, 'recall': 0.6153846153846154, 'F1': 0.63681592039801, 'ERDE_5': 0.2648698254776353, 'ERDE_50': 0.125532267078377, 'median_latency_tps': 13.0, 'median_penalty_tps': 0.046765862163709926, 'speed': 0.9532341378362901, 'latency_weighted_f1': 0.6070346748410207}


In [ ]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 3

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

In [ ]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=100, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["bayes"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)